In [1]:
import cv2
import numpy as np
from model.Models import BorderPoint
from model.Models import Matrix
from model.Models import State
from model.Models import ListCore

In [2]:
# positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
#                       [430, 100], [520, 100], [520, 150],[600, 150], [600, 200], [520, 200]
#                       , [520, 440], [650, 440], [650, 630],
#                       [210, 630], [210, 530], [70, 530], [70, 420],
#                         [150, 420], [150, 140], [50, 140], [50, 40]]
# # [ngang, dọc]
# # grid_size = (60, 100)
positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
                      [430, 100], [520, 100], [520, 150],[600, 150], [600, 220], [620, 220],
                      [620, 180], [660, 180], [660, 280], 
                      [550,280], [550,220], [520, 220], 
                      [520, 440], [650, 440], [650, 630],
                      [210, 630], [210, 530], [70, 530], [70, 420], 
                      [150, 420], [150, 140], [50, 140], [50, 40]]
positions = [[60, 60], [270, 60], [270, 270], [60, 270], [60, 60]]
grid_size = [120, 120]
positions = [[50, 40], [150, 40], [150, 120], [350, 120], 
             [350, 60], [400, 60], [400, 200], [500, 200], 
             [500, 500], [50, 500], [50, 40]]

#### DI CHUYỂN MA TRẬN

In [3]:
minx = min(np.array(positions)[:, 0])
miny = min(np.array(positions)[:, 1])
if minx < miny:
    positions = positions - minx + 4
else:
    positions = positions - miny + 4


minx = min(np.array(positions)[:, 0])
miny = min(np.array(positions)[:, 1])

if minx > miny:
    positions = [[x - minx + 4,y] for x, y in positions]
else:
    positions = [[x, y - miny + 4] for x, y in positions]

positions


[[4, 4],
 [104, 4],
 [104, 84],
 [304, 84],
 [304, 24],
 [354, 24],
 [354, 164],
 [454, 164],
 [454, 464],
 [4, 464],
 [4, 4]]

#### TẠO MA TRẬN

In [4]:
border_point = BorderPoint()
# Tạo đường biên
border_line = border_point.create_line_border_all(positions)

# Chia thành 4 đường biên tính từ tọa độ đầu vào
trung_diem = [int(np.mean(np.array(border_line)[:,0])), int(np.mean(np.array(border_line)[:,1]))]
border1, border2, border3, border4 = border_point.create_line_border(trung_diem)

# Tạo ma trận 0 & 1 & -1
min_p = (min(np.array(positions)[:,0]), min(np.array(positions)[:,1]))
max_p = (max(np.array(positions)[:,0]), max(np.array(positions)[:,1]))
mt = Matrix(max_p[0] , max_p[1])
matrix = mt.create_matrix(trung_diem, grid_size, border1, border2, border3, border4)

# Lấy các góc
corners = mt.take_all_corners()
# Cập nhật các góc có giá trị 2
matrix_have_corners = mt.add_corners_to_matrix(corners)
# Đánh dấu cho phần thịt
matrix_lable = mt.label_group_for_0()

#  Gom nhóm các góc
group = mt.getGroup(matrix_lable)

state = State(group)
# Sắp xếp các đỉnh theo chiều kim đồng hồ
states_sort = state.sort_state(state.state)


1
2
3
4 DONE !
(458, 468, 4)


#### Lấy vị trí cần check core

In [26]:
list_core = ListCore()

# Điểm tọa độ từ 
ress_min = list_core.find_state_and_group(states_sort, 6)
# Điểm tọa độ đến
ress_max = list_core.find_state_and_group(states_sort, 13)
min_max_group = [[min(np.array(ress_min)[:,0]-1), min(np.array(ress_min)[:,1]-1)], [max(np.array(ress_max)[:,0]+1), max(np.array(ress_max)[:,1]+1)]]
# min_max_group = [[min(np.array(ress_max)[:,0]-1), min(np.array(ress_max)[:,1]-1)], [max(np.array(ress_max)[:,0]+1), max(np.array(ress_max)[:,1]+1)]]

# tạo các core từ tọa độ điểm bắt đầu cho đến tọa độ kết thúc
cores = list_core.create_cores(matrix_lable, min_max_group[0], min_max_group[1])


[[3, 118], [455, 360]]


#### TRỰC QUAN HÓA

In [27]:
image = np.ones((700, 700, 3), np.uint8) * 255

for arr in matrix:
    for x,y,v,g in arr:
        x, y, v, g = int(x), int(y), int(v), int(g)
        if v == 2:
            cv2.line(image, (x,y), (x,y), (0,0,0),3)
        if v == 1:
            cv2.line(image, (x,y), (x,y), (100,100,100),1)
            
for arr in matrix:
    for x,y,v,g in arr:
        x, y, v, g = int(x), int(y), int(v), int(g)
        if v == 2:
            core = [core for core in cores if core.point == [x, y]]
            for c in core:
                if c:
                    length = 5
                    think = 2
                    if c.TOP:
                        cv2.line(image, (x,y), (x,y - length), (0,0,255),think)
                    if c.RIGHT:
                        cv2.line(image, (x,y), (x + length,y), (0,0,255),think)
                    if c.BOTTOM:
                        cv2.line(image, (x,y), (x,y + length), (0,0,255),think)
                    if c.LEFT:
                        cv2.line(image, (x,y), (x - length,y), (0,0,255),think)

while True:
    cv2.imshow('Brick', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [8]:
max(border3)

[454, 464]